In [1]:
#this julia code is to optimize a conversion product of algae to organics and how it could be supply to surrounding infrastructure for energy or related uses.
#The code import excel, categorize values to their respective variables range and constraint, and use JuMP to solve.

using JuMP
using Cbc 
using XLSX
using CSV
using DataFrames
using Glob

In [2]:
glob_file = glob("*.csv", raw"C:\Users\buing\Documents\school research things\algea research\t164 case")
tech_stoich = DataFrame(CSV.File(raw"C:\Users\buing\Documents\school research things\algea research\tech_stoich.csv"));
# supply_data = DataFrame(CSV.File(".//model excel_1//supplydata272.csv"));
#demand_data = DataFrame(CSV.File(".//model excel_1//demanddata272.csv"));
#tech_site = DataFrame(CSV.File(".//model excel_1//techsitedata272.csv"));
#name_sup = names(supply_data);
#str_s = supply_data."#"
#sup_cap = supply_data."Capacity"
#sup_length = nrow(supply_data)
#no_str_s = zeros(length(str_s),1)
#for i in 1:length(str_s)
#    no_str_s[i] = parse(Int64,chop(str_s[i], head = 1, tail = 0))
#end
#no_str_s = convert.(Int,no_str_s)
#println(no_str_s)
#str_s


#want to put it such that supply first, demand 2nd, tech 3rd. push into sot file in such order.
sort_file = []
    for i in 1:length(glob_file)
        if  occursin("sup",glob_file[i]) == true
        sort_file = push!(sort_file, glob_file[i])
        end
    end
    for i in 1:length(glob_file)
        if occursin("dem",glob_file[i]) == true
        sort_file = push!(sort_file, glob_file[i])
        end
    end
    for i in 1:length(glob_file)
        if occursin("tech",glob_file[i]) == true
        sort_file = push!(sort_file, glob_file[i])
        end
    end
#all_data = DataFrame(CSV.File(glob_file))

#julia> occursin("Hello", a)
#true

#julia> findfirst("Hello", a)
#1:5

In [3]:
#categorize supply,demand, tech to respective variables.
supply_data = DataFrame(CSV.File(sort_file[1]));
demand_data = DataFrame(CSV.File(sort_file[2]));
tech_site = DataFrame(CSV.File(sort_file[3]));

In [4]:
#introduce Cbc optimizer package. Since the problem is non-linear, atribute is set to have loglevel = 0.
m = Model(with_optimizer(Cbc.Optimizer));
set_optimizer_attribute(m, "logLevel", 0)

0

In [5]:
#cap for all things, other relevant definitions
#supply cap
sup_cap = supply_data."Capacity"
#sup_length = nrow(supply_data)
#demand cap
dem_cap = demand_data."Capacity"
#prod ID relation
prod_ID = unique(demand_data."Prod ID")
#elec = p68 algea = p69, livestock p66, CO2 = p71 P = 72. the string "p(number)" and such are products ID.
prod_stoich = innerjoin(tech_stoich,unique(view(demand_data,:,[:"Prod ID", :"Price"])), on = :"reaction ID" => :"Prod ID")
prod_price = demand_data."Price"
#tech site
tech_cap = tech_site."Capacity"

@variables(m, begin #introduce variables range for supply and demand capacity of each location. 
#In other words, these are the variables we are trying to optimize. We are figuring out the optimal supply each location should provide.
         0 <= s[i=1:nrow(supply_data)] <= sup_cap[i];   #supply variable cap
         0 <= d[i=1:nrow(demand_data)] <= dem_cap[i];   #demand variable cap
         p[i = 1:length(prod_ID)];                      #production def
    end)
#Dict_sup_id = Dict()
#for i in 1:nrow(supply_data)
#   Dict_sup_id[(i)] = Dict(s[i] => str_s[i])
#end
        

In [6]:
#technology site. same introduction as above, but for technology of converting input material to production ID above.
#each site supply/receive raw materials, and these raw materials are converted to products.
tech_cap = tech_site."Capacity"
@variable(m, 0 <= ts[i=1:nrow(tech_site)] <= tech_cap[i]); #tech ID 164 with cap
@variable(m, f_sup[i = 1:nrow(supply_data)] >= 0); #all supply to tech
@variable(m, f_dem[i = 1:nrow(demand_data)] >= 0); #all tech to dem
@variables(m, begin
    supply_cost >= 0;
    demand_revenue >= 0;
    investment_cost >= 0;
    operation_cost >= 0;
    transport_cost >= 0;
    system_profit;
    end)


In [7]:
dem_A = [1, 6, 11, 16, 21, 26, 31] #due to how the supply/demand variables were defined above, these numbers represented the start of each location.
#this is hardcoded, which is not the most efficient way.
@constraints m begin
    s .== f_sup; #where supply goes into technology. here there is only 1 technology.
    ts[1] .== sum(f_sup); #here a tech site has all supply from suppliers for techsite 164.
    p[1] <= ts[1];
    #product transformation tech
    prod_stoich."t164".*p[1] .== p;   #stoichemitric balance for  tech balance t164
    sum(s) == p[1];
    #mass balance of prod going to demand
    sum(d[dem_A]) == p[3]; # p 68
    sum(d[dem_A.+1]) == p[2]; # p69
    sum(d[dem_A.+2]) == p[4]; # p73
    sum(d[dem_A.+3]) == p[5]; # p 74
    sum(d[end-1]) == p[6];  #p 71
    sum(d[end]) == p[7];   # p 72
    d .== f_dem;
    
    supply_cost == sum((supply_data."Price") .*s);
    demand_revenue == sum(d.*prod_price);
    investment_cost == 131946409.6;
    operation_cost == 0;
    transport_cost == 0;
    
    system_profit == demand_revenue - supply_cost - investment_cost - operation_cost - transport_cost
    
end

@objective(m, Max, system_profit); #we wnat to maximize the entire system profit. This will tell each production site the optimal production they should have within constraint.

optimize!(m)

Presolve 0 (-66) rows, 0 (-102) columns and 0 (-191) elements
Optimal - objective value -1.1288673e+08
After Postsolve, objective -1.1288673e+08, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective -112886731.1 - 0 iterations time 0.012, Presolve 0.01


In [8]:
print(m)

Max system_profit
Subject to
 s[1] - f_sup[1] == 0.0
 s[2] - f_sup[2] == 0.0
 s[3] - f_sup[3] == 0.0
 s[4] - f_sup[4] == 0.0
 s[5] - f_sup[5] == 0.0
 s[6] - f_sup[6] == 0.0
 s[7] - f_sup[7] == 0.0
 ts[1] - f_sup[1] - f_sup[2] - f_sup[3] - f_sup[4] - f_sup[5] - f_sup[6] - f_sup[7] == 0.0
 0 == 0.0
 39.49119 p[1] - p[2] == 0.0
 0.039243 p[1] - p[3] == 0.0
 0.0405 p[1] - p[4] == 0.0
 0.0004983 p[1] - p[5] == 0.0
 -p[6] == 0.0
 -p[7] == 0.0
 s[1] + s[2] + s[3] + s[4] + s[5] + s[6] + s[7] - p[1] == 0.0
 d[1] + d[6] + d[11] + d[16] + d[21] + d[26] + d[31] - p[3] == 0.0
 d[2] + d[7] + d[12] + d[17] + d[22] + d[27] + d[32] - p[2] == 0.0
 d[3] + d[8] + d[13] + d[18] + d[23] + d[28] + d[33] - p[4] == 0.0
 d[4] + d[9] + d[14] + d[19] + d[24] + d[29] + d[34] - p[5] == 0.0
 d[36] - p[6] == 0.0
 d[37] - p[7] == 0.0
 d[1] - f_dem[1] == 0.0
 d[2] - f_dem[2] == 0.0
 d[3] - f_dem[3] == 0.0
 d[4] - f_dem[4] == 0.0
 d[5] - f_dem[5] == 0.0
 d[6] - f_dem[6] == 0.0
 d[7] - f_dem[7] == 0.0
 d[8] - f_dem[8] ==

In [15]:
d.*demand_data."Price"
#(value.(d)).*demand_data."Price"
value.(system_profit)

-1.12886731139396e8

In [11]:
#for i = 1:(length(supply_data."#")-2)
#demand_data[(i*5+1):((i+1)*5+1),:]
prod_stoich

,reaction ID,t164,Price
,String,Float64,Float64
1,p66,1.0,0.0
2,p68,39.4912,0.11
3,p69,0.039243,2500.0
4,p71,0.0405,0.0
5,p72,0.0004983,0.0
6,p73,0.0,362.9
7,p74,0.0,425.0


In [12]:
A = [1, 6, 11, 16, 21, 26, 31]
demand_data."#"[A.+3]
demand_data."#"[end]

"d37"